In [9]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.2 MB/s eta 0:00:00


In [1]:
# Install necessary libraries
!pip install annoy torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552447 sha256=4735957299ca4e09d3389f29188283f840844304f7ed46bb44feaeb987ded99a
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [13]:
from annoy import AnnoyIndex
import numpy as np
import torch
import faiss
from torchvision import models, transforms
from PIL import Image
from transformers import BertTokenizer, BertModel

In [3]:
# Define function to get image embeddings
def get_image_embedding(image_path):
    model = models.resnet18(pretrained=True)
    model.eval()
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = model(image_tensor).numpy()
    return features.flatten()

In [4]:
# Sample image paths
image_paths = ["jesusphoto.jpeg", "dogg.jpeg", "peacock.jpeg"]

In [5]:
# Get embeddings
embeddings = np.array([get_image_embedding(path) for path in image_paths])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 68.3MB/s]


In [15]:
# Create Annoy index
dimension = embeddings.shape[1]
index = AnnoyIndex(dimension, 'angular')
for i, embedding in enumerate(embeddings):
    index.add_item(i, embedding)
index.build(10)

True

In [19]:
# Query
query_embedding = get_image_embedding("query_image.jpeg")
nearest_neighbors = index.get_nns_by_vector(query_embedding, 2)  # Find 2 nearest neighbors

In [20]:
# Output results
print("Nearest neighbors:", nearest_neighbors)

Nearest neighbors: [2, 0]
